# Data Exploration: Federal RePORTER

This assignment is written individually by Siqi Wang and is designed by Maryah Garner

## Table of Contents
* [Load projects data](#load-data)
* [Data glimpse](#data-glimpse)
* [Columns](#columns)
* [Grouping and aggregating data](#grouping-aggregating)
* [Merge dataframes](#merge-dataframes)
* [Subsetting data](#subsetting-data)
* [Descriptive stats](#descriptive-stats)
* [Abstracts](#abstracts)
* [Checkpoint](#checkpoint)

# Assignment 2 <a class="anchor" id="checkpoint"></a>
## All text anwers are in green boxes.
### 1. Read in projects data for a year of your choice (other then 2016)
<div class="alert alert-success">I read in projects data for year 2021.</div>
    

#### Import necessary libraries <a class="anchor" id="columns"></a>

In [1]:
import pandas as pd

#### Load projects data

In [2]:
# Specify a path with the data folder
# Change "NAME" to your name as recorded on your computer
# path = 'C:/Users/NAME/PADM-GP_2505/Data/'
Path = '/Users/wsq/Desktop/Advanced Data Analytics and Evidence Building/PADM-GP_2505/Data'
#Read-in a CSV file
grants_2021 = pd.read_csv(Path + '/Projects/RePORTER_PRJ_C_FY2021_new.csv', encoding='latin-1')

#### Data glimpse

In [3]:
# We can see how many (rows, columns) there are in the dataframe by using .shape
grants_2021.shape

(82741, 46)

In [4]:
# See first 5 rows with head() function
grants_2021.head(5)

,APPLICATION_ID,ACTIVITY,ADMINISTERING_IC,APPLICATION_TYPE,ARRA_FUNDED,AWARD_NOTICE_DATE,BUDGET_START,BUDGET_END,CFDA_CODE,CORE_PROJECT_NUM,...,SERIAL_NUMBER,STUDY_SECTION,STUDY_SECTION_NAME,SUBPROJECT_ID,SUFFIX,SUPPORT_YEAR,DIRECT_COST_AMT,INDIRECT_COST_AMT,TOTAL_COST,TOTAL_COST_SUB_PROJECT
0,10595864,U54,DK,6.0,N,06/13/2022,04/01/2022,07/31/2022,NaN,U54DK106829,...,106829.0,ZDK1,Special Emphasis Panel,7612.0,NaN,7.0,42060.0,31955.0,NaN,74015.0
1,10101643,R01,DA,5.0,N,02/22/2021,03/01/2021,02/28/2022,279.0,R01DA046197,...,46197.0,ZRG1,Special Emphasis Panel,NaN,NaN,4.0,451257.0,167187.0,618444.0,NaN
2,10189622,U18,FD,5.0,N,06/08/2021,06/01/2021,05/31/2022,103.0,U18FD006442,...,6442.0,ZFD1,Special Emphasis Panel,NaN,NaN,4.0,NaN,NaN,74000.0,NaN
3,10189608,U18,FD,5.0,N,06/01/2021,06/01/2021,05/31/2022,103.0,U18FD006164,...,6164.0,ZFD1,Special Emphasis Panel,NaN,NaN,5.0,NaN,NaN,52000.0,NaN
4,10076833,R01,EY,5.0,N,01/11/2021,01/01/2021,12/31/2021,867.0,R01EY015240,...,15240.0,BVS,Biology of the Visual System Study Section,NaN,NaN,16.0,335775.0,204822.0,540597.0,NaN


#### Group by and aggregation functions
It is possible to group the dataframe by a column, and use aggregation function on them, and sort the result.

For example, we would like to know: how many NIH grants were awarded by each administering agency, Institute, or Center?

#### Create dataframe & Remane columns

In [5]:
#### Create dataframe
#### Remane columns
# calculate how many grants (unique application ids) that were awarded by each administering agency, Institute, or Center (IC_NAME)
# step1: in the groupby() method, we pass the column we want to group by
# step2: use the nunique() method to count the number of unique values (in this case, number of unique application ids by each entity)
# step3: sort the results in descending order (set the ascending parameter to False)

df_group = grants_2021.groupby('IC_NAME')['APPLICATION_ID'].nunique().sort_values(ascending=False)
df_group.head()

# Note that the aggregation function didn't return a dataframe. 
# So we have to convert it into a dataframe if we want to process it further
df_group = df_group.to_frame().reset_index()
df_group.head()

# Let's correct the columns names, this shouldn't be project_id but a number of all funded projects
df_group.rename(columns={'APPLICATION_ID':'NUMBER OF FUNDED PROJECTS'}, inplace = True)
df_group.head(5)

,IC_NAME,NUMBER OF FUNDED PROJECTS
0,NATIONAL CANCER INSTITUTE,12595
1,NATIONAL INSTITUTE OF ALLERGY AND INFECTIOUS D...,9218
2,NATIONAL INSTITUTE OF GENERAL MEDICAL SCIENCES,8647
3,"NATIONAL HEART, LUNG, AND BLOOD INSTITUTE",6962
4,NATIONAL INSTITUTE ON AGING,5960


Instead of just looking at the total number of projects funded by each intity, you might also want to know the sum ot the total cost of these projects

In [6]:
# Convert scientific notation to a full float
pd.set_option('display.float_format', '{:.2f}'.format)

In [7]:
# calculate how the sum of the total costs for each administering agency, Institute, or Center (IC_NAME)
# step1: in the groupby() method, we pass the column we want to group by
# step2: use the sum() method to add together the total costs (in this case, number of unique application ids by each entity)
# step3: sort the results in descending order (set the ascending parameter to False)

Cost = grants_2021.groupby('IC_NAME')['TOTAL_COST'].sum().sort_values(ascending = False)

# step3: convert into a dataframe and reset index

Cost = Cost.to_frame().reset_index()
Cost.head()

,IC_NAME,TOTAL_COST
0,NATIONAL INSTITUTE OF ALLERGY AND INFECTIOUS D...,6604315172.00
1,NATIONAL CANCER INSTITUTE,6053834866.00
2,"NATIONAL HEART, LUNG, AND BLOOD INSTITUTE",4492970633.00
3,NATIONAL INSTITUTE ON AGING,3377310191.00
4,NATIONAL INSTITUTE OF GENERAL MEDICAL SCIENCES,3010854665.00


Other useful aggregation functions are: sum(): sum, mean(): average, agg(): use a python dictionary to specify aggregation function based on each column

#### Merge Dataframes <a class="anchor" id="merge-dataframes"></a>
Pandas provides an ability to merge (join) two datasets together. You can store the results in a new dataframe. There are different ways of merging data: left, right, outer, inner (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html).

Join together the two dataframes we just created suinf the common identifier (`IC_NAME`)


In [8]:
# Merge the first dataframe "df_group" with the count of projects per funding entity with the Cost datafrome
# how='inter' means use the intersection of the same project ids between the two dataframes

merge_df = pd.merge(df_group, Cost, on='IC_NAME', how = 'inner')
merge_df.head(5)

,IC_NAME,NUMBER OF FUNDED PROJECTS,TOTAL_COST
0,NATIONAL CANCER INSTITUTE,12595,6053834866.00
1,NATIONAL INSTITUTE OF ALLERGY AND INFECTIOUS D...,9218,6604315172.00
2,NATIONAL INSTITUTE OF GENERAL MEDICAL SCIENCES,8647,3010854665.00
3,"NATIONAL HEART, LUNG, AND BLOOD INSTITUTE",6962,4492970633.00
4,NATIONAL INSTITUTE ON AGING,5960,3377310191.00


### 2. Subset the data for a specific administering agency, Institute, or Center (IC)
<div class="alert alert-success">I subseted the data for National Cancer Institute(NCI).</div>

In [9]:
merge_df2 = pd.merge(df_group, Cost, on='IC_NAME', how='inner')
merge_df2.head(2)

,IC_NAME,NUMBER OF FUNDED PROJECTS,TOTAL_COST
0,NATIONAL CANCER INSTITUTE,12595,6053834866.00
1,NATIONAL INSTITUTE OF ALLERGY AND INFECTIOUS D...,9218,6604315172.00


In [10]:
# conditional subsetting: put the conditional statement within the square brackets 
# the conditional statement here is that we want the IC_NAME to be NATIONAL CANCER INSTITUTE. 

df_NCI = grants_2021[grants_2021['IC_NAME'] == 'NATIONAL CANCER INSTITUTE']
df_NCI.head(2)

,APPLICATION_ID,ACTIVITY,ADMINISTERING_IC,APPLICATION_TYPE,ARRA_FUNDED,AWARD_NOTICE_DATE,BUDGET_START,BUDGET_END,CFDA_CODE,CORE_PROJECT_NUM,...,SERIAL_NUMBER,STUDY_SECTION,STUDY_SECTION_NAME,SUBPROJECT_ID,SUFFIX,SUPPORT_YEAR,DIRECT_COST_AMT,INDIRECT_COST_AMT,TOTAL_COST,TOTAL_COST_SUB_PROJECT
18,10406126,P30,CA,3.00,N,09/23/2021,09/01/2021,11/30/2021,397.00,P30CA033572,...,33572.00,NaN,NaN,NaN,S2,38.00,135722.00,83904.00,219626.00,NaN
29,10246936,U01,CA,5.00,N,08/30/2021,09/01/2021,08/31/2022,353.00,U01CA239258,...,239258.00,ZCA1,Special Emphasis Panel,NaN,NaN,4.00,529513.00,107208.00,636455.00,NaN


### 3. What are the top 5 organizations (by number of projects) who have received funding from this entity? 
<div class="alert alert-success">

The top 5 organizations are:
<p>1. DIVISION OF BASIC SCIENCES - NCI<p/>           
<p>2. UNIVERSITY OF TX MD ANDERSON CAN CTR<p/>       
<p>3. DANA-FARBER CANCER INST<p/>                    
<p>4. SLOAN-KETTERING INST CAN RESEARCH<p/>       
<p>5. UNIVERSITY OF CALIFORNIA, SAN FRANCISCO</div>

In [11]:
df_NCI_ORG = df_NCI.groupby('ORG_NAME')['APPLICATION_ID'].nunique().sort_values(ascending=False)
df_NCI_ORG.head()

df_NCI_ORG = df_NCI_ORG.to_frame().reset_index()
df_NCI_ORG.rename(columns={'APPLICATION_ID':'NUMBER OF FUNDED PROJECTS'}, inplace = True)
df_NCI_ORG.head()

,ORG_NAME,NUMBER OF FUNDED PROJECTS
0,DIVISION OF BASIC SCIENCES - NCI,735
1,UNIVERSITY OF TX MD ANDERSON CAN CTR,371
2,DANA-FARBER CANCER INST,349
3,SLOAN-KETTERING INST CAN RESEARCH,327
4,"UNIVERSITY OF CALIFORNIA, SAN FRANCISCO",291


### 4. How many projects were funded by each of the top 5 organizations, and what is the total cost? 
<div class="alert alert-success">
<p> <strong>1.DIVISION OF BASIC SCIENCES</strong> <p>NCI: 735 projects funded, total cost: 945249254.00
<p> <strong>2.UNIVERSITY OF TX MD ANDERSON CAN CTR</strong> <p>371 projects funded, total cost:172444889.00
<p> <strong>3.DANA-FARBER CANCER INST</strong> <p>349 projects funded, total cost: 159604697.00
<p> <strong>4.SLOAN-KETTERING INST CAN RESEARCH</strong> <p>327 projects funded, total cost:186852467.00
<p> <strong>5.UNIVERSITY OF CALIFORNIA, SAN FRANCISCO</strong> <p>291 projects funded, total cost: 722921214.00 </div>

In [12]:
# calculate how the sum of the total costs for each administering agency, Institute, or Center (IC_NAME)
# step1: in the groupby() method, we pass the column we want to group by
# step2: use the sum() method to add together the total costs (in this case, number of unique application ids by each entity)
# step3: sort the results in descending order (set the ascending parameter to False)

Cost_ORG = grants_2021.groupby('ORG_NAME')['TOTAL_COST'].sum().sort_values(ascending = False)

# step3: convert into a dataframe and reset index

Cost_ORG = Cost_ORG.to_frame().reset_index()
Cost_ORG.head()

,ORG_NAME,TOTAL_COST
0,DIVISION OF BASIC SCIENCES - NCI,945249254.00
1,NATIONAL INSTITUTE OF ALLERGY AND INFECTIOUS D...,856340580.00
2,JOHNS HOPKINS UNIVERSITY,836216949.00
3,NEW YORK UNIVERSITY SCHOOL OF MEDICINE,816250852.00
4,DUKE UNIVERSITY,739437056.00


In [13]:
merge_NCI = pd.merge(df_NCI_ORG, Cost_ORG, on='ORG_NAME', how='inner')
merge_NCI.rename(columns={'APPLICATION_ID':'NUMBER OF FUNDED PROJECTS'}, inplace = True)
merge_NCI.head()

,ORG_NAME,NUMBER OF FUNDED PROJECTS,TOTAL_COST
0,DIVISION OF BASIC SCIENCES - NCI,735,945249254.00
1,UNIVERSITY OF TX MD ANDERSON CAN CTR,371,172444889.00
2,DANA-FARBER CANCER INST,349,159604697.00
3,SLOAN-KETTERING INST CAN RESEARCH,327,186852467.00
4,"UNIVERSITY OF CALIFORNIA, SAN FRANCISCO",291,722921214.00


### 5. Who are the top five PIs from the organizations with the most projects, and how many projects were they the PI for? 
<div class="alert alert-success">
<p> <strong>1. GLIMCHER, LAURIE HOLLIS</strong> with 12 projects.<p>
<p> <strong>2. VAN DEN BRINK, MARCEL R M</strong> with 9 projects.<p>
<p> <strong>3. SHARAN, SHYAM , DAN</strong> with 9 projects.<p>
<p> <strong>4. DECAPRIO, JAMES A</strong> with 8 projects.<p>
<p> <strong>5. SINGER, ALFRED</strong> with 8 projects.</div>

In [14]:
df_cost_ORG = df_NCI[df_NCI['ORG_NAME'].isin(['DIVISION OF BASIC SCIENCES - NCI', 
                                        'SLOAN-KETTERING INST CAN RESEARCH',
                                        'DANA-FARBER CANCER INST',
                                        'UNIVERSITY OF TX MD ANDERSON CAN CTR','UNIVERSITY OF CALIFORNIA, SAN FRANCISCO'])]
df_cost_ORG = df_cost_ORG.groupby(['PI_IDS','PI_NAMEs','ORG_NAME'])['APPLICATION_ID'].nunique().sort_values(ascending=False).to_frame().reset_index()
df_cost_ORG.rename(columns={'APPLICATION_ID':'Number of Funded Projects'}, inplace = True)
df_cost_ORG.head(5)

,PI_IDS,PI_NAMEs,ORG_NAME,Number of Funded Projects
0,1871771;,"GLIMCHER, LAURIE HOLLIS;",DANA-FARBER CANCER INST,12
1,6623254;,"VAN DEN BRINK, MARCEL R M;",SLOAN-KETTERING INST CAN RESEARCH,9
2,6809564;,"SHARAN, SHYAM ;",DIVISION OF BASIC SCIENCES - NCI,9
3,1941096;,"DECAPRIO, JAMES A;",DANA-FARBER CANCER INST,8
4,9692601;,"SINGER, ALFRED ;",DIVISION OF BASIC SCIENCES - NCI,8
